#**Imports**#

In [6]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm
import time
import pandas as pd
import re
import hashlib
import numpy as np

#**Load Data**#

In [350]:
url = 'https://www.kijiji.it/offerte-di-lavoro/offerta/informatica-e-web/'

In [351]:
def get_data_from_url(url, num_pages):

  new_url = url

  with open('/tmp/output.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(['Title', 'Description', 'Location', 'Publication Date', 'URL'])

    for i in tqdm(range(1, num_pages+1)):
      if i > 1:
        new_url = url + "?p=" + str(i)

      r = requests.get(new_url)
      soup = BeautifulSoup(r.content)
      g_data = soup.find_all("div", {"class": "item-content"})
      for item in g_data:
        tsv_writer.writerow([item.contents[1].find_all("a", {"class": "cta"})[0].text.strip(),
                             item.contents[3].text, 
                             item.contents[7].text, item.contents[9].text,
                             item.contents[1].find_all("a", {"class": "cta"})[0].get('href')])
      time.sleep(0.75)

#152 is the number of pages with job announcements on the kijiji web page in the Informatica/Grafica/Web sector     
get_data_from_url(url, 152)

100%|██████████| 152/152 [04:10<00:00,  1.65s/it]


In [352]:
tsv_file = open("/tmp/output.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")

for i, row in enumerate(read_tsv):
  print(row)
  if i == 5:
    break

['Title', 'Description', 'Location', 'Publication Date', 'URL']
['Editor Premiere - videografico-a After Effects', 'Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla struttura; si richiede 1) conoscenza approfondita di Premiere e After Effects, basilare conoscenza di Davinci, residenza entro 15 km da Novara\nSe interessati e in possesso di tutti i requisiti inviare un curriculum a Job AT giuseppegalliano.it e link a proprio show reel.', 'Novara', '31 agosto, 19:02', 'https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videografico-a-after-effects/159299183']
['Analista software', 'Hai ottime capacità di analisi e attitudine al problem solving\' Per società operante nel settore informatico ricerchiamo un Analista che si occupi di analizzare i requisiti e le esigenze di business per produrre le specifiche tecniche e i progetti.In collaborazione con i soci ti occuperai della raccolta di tutte le informazion

In [354]:
df = pd.read_csv('/tmp/output.tsv', delimiter = '\t')
df.count()

Title               3187
Description         3187
Location            3187
Publication Date    3187
URL                 3187
dtype: int64

In [355]:
df.head()

,Title,Description,Location,Publication Date,URL
0,Editor Premiere - videografico-a After Effects,Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla...,Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videogra...
1,Analista software,Hai ottime capacità di analisi e attitudine al problem solving' Per società operante nel settore...,Belluno,"Oggi, 04:10",https://www.kijiji.it/annunci/offerta/belluno-annunci-belluno/analista-software/167793348
2,Operaio per car wrapping e antifurti,"\n\nClicca sul link sottostante ""sito web"" per inviarci la tua candidatura.",Napoli,"Oggi, 04:10",https://www.kijiji.it/annunci/offerta/napoli-annunci-napoli/operaio-per-car-wrapping-e-antifurti...
3,Figura professinale ingegnere /architetto,"\n\nClicca sul link sottostante ""sito web"" per inviarci la tua candidatura.",Torino,"Oggi, 04:09",https://www.kijiji.it/annunci/offerta/torino-annunci-torino/figura-professinale-ingegnere-archit...
4,Neodiplomati/ laurea breve in ingegneria o architettura,Oggi Lavoro Srl - Filiale di Tradate seleziona per azienda cliente sita a Ternate NEODIPLOMATI/ ...,Varese,"Oggi, 04:09",https://www.kijiji.it/annunci/offerta/varese-annunci-varese/neodiplomati-laurea-breve-in-ingegne...


Analysis will be done based on the **"Description"** field of the DataFrame, as it is suggested in the assignment. 

Therefore, we **clean up data** from the announcements which contain reference to the web-page instead of the description, because they are not really duplicates of the same announcement, but different announcements without description.

In [356]:
df_with_description = df[df['Description'] != '\n\nClicca sul link sottostante "sito web" per inviarci la tua candidatura.']
df_with_description.count()

Title               1662
Description         1662
Location            1662
Publication Date    1662
URL                 1662
dtype: int64

# **Hash Functions**

A piece of code provided in the assignment, but with the corrected encoding part (row №6).

In [357]:
def hashFamily(i):
  resultSize = 8
  maxLen = 20 
  salt = str(i).zfill(maxLen)[-maxLen:]
  def hashMember(x):
    return int(hashlib.sha1(x.encode('utf-8') + salt.encode('utf-8')).hexdigest()[-resultSize:], 16)
  return hashMember

# **Shingling**

##**Kind of Shingling**

*Book Def*. If document is a string of characters, then **$k$-shingle** for a document is a substring of length $k$ found within the document.

*Note:* It's possible to work on shingles built from words or from characters. Given task assignment suggests to use exactly **characters shingling**.

##**Preprocessing**

*Suggestion from the book:* There are several options regarding how white space (blank, tab, newline, etc.) is treated. It probably makes sense to replace any sequence of one or more white-space characters by a single blank. That way, we distinguish shingles that cover two or more words from those that do not.

In our particular task assignment we consider strings, contained in the **Description** field of DataFrame as documents. After implementing pre-processing step of the Problem 1 of the current homework, it is known that in our case "sequence of one or more white-space characters" could be represented either by `\n`, or by single blank.

Therefore, everything that is needed to be done in order to follow suggestion from the book is to **replace `\n` with a single blank**.

Apart from that, amongst pre-processing steps only **lowercasing** is applied: we don't want to remove punctuation/specific characters, because they bring information that might be useful for distinguishing between shingles, we don't manipulate tokens, because the work is arranged on the character level.

##**CLASS DESCRIPTION**
*Assignment:* Implement a class that, given a document, creates its set of character shingles of some length $k$.

- **INPUT:** A DOCUMENT AS A STRING AND $k$
- LOWERCASING
- REPLACING `\n`
- DOUBLE BLANKS REMOVAL
- SPLITTING INTO SHINGLES
- **OUTPUT:** A DOCUMENT AS A SET OF SHINGLES

In [358]:
class Shingling:

    def __init__(self, doc: str, k: int = 10):
        self.doc = doc
        self.k = k #By default we will use shingles of length 10 characters

    def preprocess(self):
        self.doc = self.doc.lower()
        self.doc = re.sub('\n', ' ', self.doc)
        self.doc = re.sub(' +', ' ', self.doc)
    
    def split(self):
        self.preprocess()
        self.s = set([self.doc[i:i+self.k] for i in range(0, len(self.doc) - self.k + 1)])
        return self.s

*Assignment:* Then represent the document as the set of the hashes of the shingles, for some hash function.

In [359]:
def hashing_shingles(doc):
  sh = Shingling(doc)
  s = sh.split()
  h = hashFamily(1) #1st member of the family (could be chosen with the random seed instead of 1)
  hashed_s = set([h(shingle) for shingle in s])
  return hashed_s

##**Example**



In [360]:
doc = 'Editor\n video\n editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla struttura'
sh = Shingling(doc)
s = sh.split()
hs = hashing_shingles(doc)
print("INPUT STRING:\n", doc)
print("\nLENGTH OF SHINGLE SET AND LENGTH OF HASHED SHINGLE SET\n", (len(s), len(hs))) #To check that didn't change
print("\nEXAMPLE FOR 5 SHINGLES IN BOTH SETS\n", list(zip(list(s)[:5], list(hs)[:5])))

INPUT STRING:
 Editor
 video
 editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla struttura

LENGTH OF SHINGLE SET AND LENGTH OF HASHED SHINGLE SET
 (106, 106)

EXAMPLE FOR 5 SHINGLES IN BOTH SETS
 [('o editor v', 133963275), ('editor vid', 825021965), ('entata esp', 822642194), (' struttura', 1619594258), (' alla stru', 3601244693)]


# **Minwise Hashing**

*Assignment:* Implement a class, that given a collection of sets of objects (e.g., strings, or numbers), creates a minwise hashing based signature for each set.

So, first of all, let's **create** a collection for the class **input**: collection of sets of strings $-$ sets of $k$-shingles.


In [361]:
# create a collection of sets for class input
description_strings = df_with_description['Description']
description_sets = description_strings.apply(lambda x: Shingling(x, k=10).split())
description_sets[:5]

0    {emiere e a, residenza ,  struttura, entata esp,  alla stru, ndita e do, enza si ri, siti invia,...
1    {ettore: in, 679). facs, ità di ana, ifico / te, colaurea t, formazioni, isponibile, ttazione d,...
4    {ratto: inz, sito l'inf, 003). aut., nza di ske, tazione, r, a. e' pref,  i candida, previsto u,...
5    {in progett, niche, cat,  mansioni , l.903/77) , declinazio,  (art.13, ,  azienda c, ionale;-es,...
6    {nte gli in, ni:program, to informa, a e di lav, rtimento r,  sebbene n, n/una adde, eader tra ,...
Name: Description, dtype: object

##**CLASS DESCRIPTION**


$t$ $-$ signature length, by default is set equal to $100$.

- **INPUT:** COLLECTION OF SETS OF STRINGS AND $t$
- REPEAT $\forall$ SETS IN COLLECTION
> - REPEAT $\forall$ $t$ HASH FUNCTIONS
>> - COMPUTE HASH FUNCTION $\forall$ SHINGLES IN SET
>> - FIND MINIMUM
>> - MINIMUM BECOMES AN ELEMENT OF THE SIGNATURE VECTOR
- **OUTOUT:** COLLECTIONS OF SIGNATURE VECTORS

In [362]:
class Minwise:

    def __init__(self, collection: pd.Series, t: int = 100):
      self.collection = collection
      self.t = t

    def create_signature(self):
        self.signature = pd.DataFrame(data=None)
        for i in tqdm(range(1, self.t+1)):
          h_i = hashFamily(i)
          f_i = self.collection.apply(lambda shingle_set: min(set([h_i(shingle) for shingle in shingle_set])))
          self.signature = pd.concat([self.signature, f_i], axis=1)
          self.signature = self.signature.rename(columns={'Description': 'f_{}'.format(i)})
        return self.signature


In [363]:
m = Minwise(description_sets)
sig_df = m.create_signature()

100%|██████████| 100/100 [04:47<00:00,  2.87s/it]


In [364]:
signature_vectors = sig_df.values
signature_vectors[:1]

array([[26345623, 38050713, 14601143,  4072488, 12636253,  1374495,
         3767497, 30256053,  8388411,  6990399,  6299464, 11206475,
        15030230,  5133621, 38020840, 29994485,  8252367, 19923526,
         1143702,  3719829,  1680935, 11895970, 62358787, 97293210,
        27355082,  6977510,  2602742, 18295243,  8487226,  9983296,
        27715896,  1311768, 12890886, 11470547,   378705, 90170337,
         1193263,  1710898,   244541,  7887395, 11085649,  8959964,
         7243119, 31827677,  8167119, 10291217,  7595086, 47243450,
        25394902,  7967445, 14611815,  4624032,  8033487, 10773470,
        10816685, 12977785,  6499529, 23824638, 11770429,   268774,
        27585376, 14722289,  2839895,  5715007,  6293604, 12604456,
         3060580,  8889701,   836107, 11702635,  3254113, 17504190,
         8211964, 78083229, 31836250,  4603064,  9799707, 25639016,
         2426216,   375618, 15173631, 10404544, 15187937, 15540584,
        25649280, 14300579,  4317217,  4243054, 

# **LSH**


*Assignment:* Implement a class that implements the locally sensitive hashing (LSH) technique, so that, given a collection of minwise hash signatures of a set of documents, it finds the all the documents pairs that are near each other.

##**Threshold**

We will say that two announcements are near duplicates if the Jaccard coefficient of their shingle sets is at least $80%$. 

Therefore, we **choose a threshold** $s$ that defines how similar documents have to be in order for them to be regarded as a desired "similar pair" equal to $0.8$.

##**$r$ and $b$**

For a fised $t = 100$ a number of bands $b$ and a number of rows $r$ are picked such that $br = t$, and the threshold $s$ $\approx$ $\frac{1}{b}^\frac{1}{r}$.

$b = 10, r = 10$ are optimal for this case, although in fact they produce a bit lower threshold $s \approx 0.794$, which penalizes **false negatives** (therefore, false positives are vice versa expected).

##**CLASS DESCRIPTION**

- **INPUT:** SIGNATURE MATRIX AS A PANDAS DATAFRAME (TRANSPOSED), NUMBER OF BANDS $b$ AND DESIRED THRESHOLD $s$
- CREATE AN EMPTY SET THAT WILL KEEP 2-ELEMENT TUPLES $-$ INDICES OF CANDIDATE PAIR OF DOCUMENTS
- DIVIDE THE SIGNATURE MATRIX INTO $b$ BANDS CONSISTING OF $r$ ROWS EACH
- REPEAT $\forall$ BAND
> - CREATE AN EMPTY DICTIONARY FOR THIS BAND WHERE KEYS WILL BE HASHES AND VALUES WILL BE LISTS OF INDICES OF ALL THE DOCUMENTS THAT WERE HASHED TO THIS HASH FOR THE CURRENT BAND
> - COMPUTE A HASH FUNCTION THAT TAKES VECTOR OF $r$ INTEGERS (THE PORTION OF ONE COLUMN WITHIN CURRENT BAND) AND HASHES THEM
> - UPDATE THE BAND DICTIONARY AND IF SOME NEW CANDIDATE PAIR APPEARS, UPDATE SET OF CANDIDATES
- FILTER SET OF CANDIDATES (COMPARISON OF THE ESTIMATED JACCARD SIMILARITY WITH A GIVEN THRESHOLD)
- **OUTPUT:** SET OF TUPLES, EACH TUPLE $-$ INDICES OF PAIR OF DOCUMENTS THAT ARE NEAR EACH OTHER




In [365]:
class LSH:

    def __init__(self, collection: pd.DataFrame, b: int = 10, s: float = 0.8):
      self.collection = collection
      self.t = self.collection.shape[1]
      self.b = b
      self.r = int(self.t / self.b)
      self.h = hashFamily(1)
      self.candidates = set()
      self.s = s
      print("COLLECTION:\n", self.collection)

    def find_candidates(self):
        print("\nCandidates search has started.")
        for band_start in tqdm(range(0, self.t, self.r)):
        
          band_names = self.collection.columns[band_start:band_start+self.r]
          band = self.collection[band_names]

          band_dict = dict()
          for (doc_id, doc) in zip(band.index, band.values):
            doc_hash = self.h(str(doc))
            if doc_hash not in list(band_dict.keys()):
              l = list()
              l.append(doc_id)
              band_dict[doc_hash] = l
            else:
              for pair_candidate in band_dict[doc_hash]:
                self.candidates.add(tuple(sorted([pair_candidate, doc_id])))
              band_dict[doc_hash].append(doc_id)

        return self.check_jaccard()

    def check_jaccard(self):
      self.output = set()
      print("\nCandidates check has started.")
      for (doc_id1, doc_id2) in tqdm(self.candidates):
        if (self.collection[self.collection.index==doc_id1].values==self.collection[self.collection.index==doc_id2].values).sum()/self.t >= self.s:
          self.output.add((doc_id1, doc_id2))
      return self.output      

In [366]:
lsh = LSH(sig_df)
lsh_duplicates = lsh.find_candidates()

COLLECTION:
            f_1       f_2       f_3  ...      f_98      f_99     f_100
0     26345623  38050713  14601143  ...  12621437  19284372  11911811
1      1744654   2620362   4386901  ...   2611723   6612311   3222934
4      4315546    373729   1594827  ...    665243   5136030   6214771
5      1225647    373729   1503761  ...   7532408   1660821   3823569
6       647817   2481720    793272  ...  11357568   5292696   5992101
...        ...       ...       ...  ...       ...       ...       ...
3179   1573293   3271937  15739850  ...  32873585    777998   8734718
3180   4315546    373729   2771673  ...   2643912    838145   2272727
3182   3588297   2285455    249993  ...   2611723   3687878  10928945
3183   1573293   1768218   9961987  ...    665243  14734767    450654
3186   2171580    373729   2585175  ...    651198   1270011   3823569

[1662 rows x 100 columns]

Candidates search has started.


100%|██████████| 10/10 [00:01<00:00,  5.76it/s]



Candidates check has started.


100%|██████████| 14085/14085 [00:09<00:00, 1443.35it/s]


##**LSH Time**

It takes $10s$ to derive a set of near-duplicate pairs with LSH. 

Note, that Minwise step took a long while ($5m$) to get signature vectors, but if we estimate LSH only, it's very fast. Meanwhile, Minwise step time depends on the desired length of signature vectors and could be shortened (or more advanced arrangement of the Minwise class internal computations could probably improve the result even with a current desired length of signature vectors).

# **Testing**

*Assignment:* To test the LSH algorithm, also implement a class that given the shingles of each of the documents, finds the nearest neighbors by comparing all the shingle sets with each other.

##**CLASS DESCRIPTION**

- **INPUT:** COLLECTION OF SETS OF STRINGS ($k$-SHINGLES)
- CREATE AN EMPTY SET THAT WILL KEEP 2-ELEMENT TUPLES $-$ INDICES OF CANDIDATE PAIR OF DOCUMENTS
- REPEAT $\forall$ SETS IN COLLECTION
> - COMPUTE JACCARD SIMILARITY BETWEEN CURRENT SET AND ALL SETS IN THE COLLECTION WITH WHICH IT HASN'T BEEN COMPARED YET
> - IF COMPUTED SIMILARITY IS ABOVE THE GIVEN THRESHOLD, ADD INDICES OF CURRENT PAIR TO THE OUTPUT SET
- **OUTPUT:** SET OF TUPLES, EACH TUPLE $-$ INDICES OF PAIR OF DOCUMENTS THAT ARE NEAR EACH OTHER

In [367]:
class DirectComparison:

    def __init__(self, collection: pd.Series, s: float = 0.8):
      self.collection = collection
      self.s = s
      self.output = set()

    def check_jaccard(self):

      for (doc_id1, doc1) in tqdm(zip(self.collection.index, self.collection.values)):
        following = self.collection[self.collection.index > doc_id1]
        for (doc_id2, doc2) in zip(following.index, following.values):
          if len(doc1.intersection(doc2)) / len(doc1.union(doc2)) >= self.s:
            self.output.add((doc_id1,doc_id2))
      return self.output        

In [368]:
dc = DirectComparison(description_sets)
dc_duplicates = dc.check_jaccard()

1662it [02:47,  9.93it/s] 


##**Direct Comparison Time**

It takes around $3m$ to find nearest neighbors by comparing all the shingle sets with each other, which is expectedly long.

##**Number of duplicates**

*Assignment:* Report the number of duplicates found in both cases, and the size of the intersection.**Текст, выделенный полужирным шрифтом**

As we can see, LSH found almost all the pairs correctly.

In [369]:
print("The number of duplicates found by LSH is equal to ", len(lsh_duplicates), 
      "\nThe number of duplicates found by direct comparison is equal to ", len(dc_duplicates), 
      "\nSize of intersection is equal to ", len(lsh_duplicates.intersection(dc_duplicates)))

The number of duplicates found by LSH is equal to  13911 
The number of duplicates found by direct comparison is equal to  13910 
Size of intersectio is equal to  13900


##**Print Examples**

Let's display some near-duplicates announcements.

In [371]:
# The desired number of examples to be set
def display_announcements(duplicates, num_examples):
  for i in range(num_examples):
    (doc_id1, doc_id2) = duplicates.pop()
    print((doc_id1, doc_id2))
    print("-"*200)
    pd.set_option('display.max_colwidth', 100)
    df1 = df[df.index==doc_id1]
    df2 = df[df.index==doc_id2]
    display(df1.append(df2))
    print("-"*200)

In [372]:
display_announcements(lsh_duplicates, 3)

(1386, 2184)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Title,Description,Location,Publication Date,URL
1386,Editor Premiere - videografico-a After Effects,Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla...,Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videogra...
2184,Editor Premiere - videografico-a After Effects,Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla...,Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videogra...


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
(2226, 3040)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Title,Description,Location,Publication Date,URL
2226,Editor Premiere - videografico-a After Effects,Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla...,Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videogra...
3040,Editor Premiere - videografico-a After Effects,Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla...,Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videogra...


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
(840, 1848)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Title,Description,Location,Publication Date,URL
840,Editor Premiere - videografico-a After Effects,Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla...,Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videogra...
1848,Editor Premiere - videografico-a After Effects,Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla...,Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videogra...


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [373]:
display_announcements(dc_duplicates, 3)

(1386, 2184)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Title,Description,Location,Publication Date,URL
1386,Editor Premiere - videografico-a After Effects,Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla...,Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videogra...
2184,Editor Premiere - videografico-a After Effects,Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla...,Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videogra...


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
(2226, 3040)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Title,Description,Location,Publication Date,URL
2226,Editor Premiere - videografico-a After Effects,Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla...,Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videogra...
3040,Editor Premiere - videografico-a After Effects,Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla...,Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videogra...


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
(840, 1848)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,Title,Description,Location,Publication Date,URL
840,Editor Premiere - videografico-a After Effects,Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla...,Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videogra...
1848,Editor Premiere - videografico-a After Effects,Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla...,Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videogra...


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
